In [1]:
#!pip install yfinance
#!pip install yahooquery
#from yahooquery import Screener
#import yfinance as yf   
import pandas as pd
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,LSTM
from tensorflow.keras.layers import Dropout
#from sklearn.linear_model import LogisticRegression
#from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
#from sklearn.preprocessing import MinMaxScaler,StandardScaler
#from sklearn.svm import SVC
#from sklearn.naive_bayes import GaussianNB
#from sklearn.neighbors import NearestCentroid
#from sklearn.ensemble import GradientBoostingClassifier
#from sklearn.neighbors import KNeighborsClassifier
import tensorflow as tf
#from sklearn.tree import DecisionTreeClassifier
#from sys import getsizeof
#from datetime import datetime
#import random 

In [2]:
clmns = [ '12 Open', '12 High', '12 Low', '12 Close', '12 Volume', '11 Open', '11 High', '11 Low', '11 Close', '11 Volume', '10 Open', '10 High', '10 Low', '10 Close', '10 Volume', '9 Open', '9 High', '9 Low', '9 Close', '9 Volume', '8 Open', '8 High', '8 Low', '8 Close', '8 Volume', '7 Open', '7 High', '7 Low', '7 Close', '7 Volume', '6 Open', '6 High', '6 Low', '6 Close', '6 Volume', '5 Open', '5 High', '5 Low', '5 Close', '5 Volume', '4 Open', '4 High', '4 Low', '4 Close', '4 Volume', '3 Open', '3 High', '3 Low', '3 Close', '3 Volume', '2 Open', '2 High', '2 Low', '2 Close', '2 Volume', '1 Open', '1 High', '1 Low', '1 Close', '1 Volume',"suggestion"]
clmnsws = ['12 Open', '12 High', '12 Low', '12 Close', '12 Volume', '11 Open', '11 High', '11 Low', '11 Close', '11 Volume', '10 Open', '10 High', '10 Low', '10 Close', '10 Volume', '9 Open', '9 High', '9 Low', '9 Close', '9 Volume', '8 Open', '8 High', '8 Low', '8 Close', '8 Volume', '7 Open', '7 High', '7 Low', '7 Close', '7 Volume', '6 Open', '6 High', '6 Low', '6 Close', '6 Volume', '5 Open', '5 High', '5 Low', '5 Close', '5 Volume', '4 Open', '4 High', '4 Low', '4 Close', '4 Volume', '3 Open', '3 High', '3 Low', '3 Close', '3 Volume', '2 Open', '2 High', '2 Low', '2 Close', '2 Volume', '1 Open', '1 High', '1 Low', '1 Close', '1 Volume']
def read_syms_from_txt():  
  with open("syms.txt","r") as f:
    lines = f.readlines()
    f.close()
  lst = []
  for line in lines:
    line = line.strip()
    lst.append(line)
  symbols = lst
  return symbols
def get_crypto_syms():
   #'all_cryptocurrencies_au','all_cryptocurrencies_ca','all_cryptocurrencies_eu','all_cryptocurrencies_gb','all_cryptocurrencies_in',
   screens = [
   'all_cryptocurrencies_us','all_cryptocurrencies_au','all_cryptocurrencies_ca','all_cryptocurrencies_eu','all_cryptocurrencies_gb','all_cryptocurrencies_in',]
   s = Screener()
   symbols = []
   for i in screens:
      data = s.get_screeners(i, count=250)
      dicts = data[i]['quotes']
      syms = [d['symbol'] for d in dicts]
      for sym in syms:
        symbols.append(sym)
   #print(len(symbols))
   #pieces = 15
   # new_arrays = np.array_split(symbols, pieces)
   return symbols
def spliting(data):
  X = data.drop(["suggestion"],axis=1)
  y = data["suggestion"]
  xTrain, xTest, yTrain, yTest = train_test_split(X, y, test_size=0.1)
  print(xTrain.shape,end=" ")
  print(yTrain.shape)
  print(xTest.shape,end=" ")
  print(yTest.shape)
  return xTrain, xTest, yTrain, yTest
def extract_data(df):
    rows = []
    for each in range(13,df.shape[0]-1):
        sugg = 0
        if df[each][3] > df[each][0]:
          sugg = 1
        row = [
                df[each-12][0],
                df[each-12][1],
                df[each-12][2],
                df[each-12][3],
                df[each-12][4],
                df[each-11][0],
                df[each-11][1],
                df[each-11][2],
                df[each-11][3],
                df[each-11][4],
                df[each-10][0],
                df[each-10][1],
                df[each-10][2],
                df[each-10][3],
                df[each-10][4],
                df[each-9][0],
                df[each-9][1],
                df[each-9][2],
                df[each-9][3],
                df[each-9][4],
                df[each-8][0],
                df[each-8][1],
                df[each-8][2],
                df[each-8][3],
                df[each-8][4],
                df[each-7][0],
                df[each-7][1],
                df[each-7][2],
                df[each-7][3],
                df[each-7][4],
                df[each-6][0],
                df[each-6][1],
                df[each-6][2],
                df[each-6][3],
                df[each-6][4],
                df[each-5][0],
                df[each-5][1],
                df[each-5][2],
                df[each-5][3],
                df[each-5][4],
                df[each-4][0],
                df[each-4][1],
                df[each-4][2],
                df[each-4][3],
                df[each-4][4],
                df[each-3][0],
                df[each-3][1],
                df[each-3][2],
                df[each-3][3],
                df[each-3][4],
                df[each-2][0],
                df[each-2][1],
                df[each-2][2],
                df[each-2][3],
                df[each-2][4],
                df[each-1][0],
                df[each-1][2],
                df[each-1][1],
                df[each-1][3],
                df[each-1][4],
                sugg
        ]
        rows.append(row)
    return rows
def row_scaler(df):
  scaler = MinMaxScaler(feature_range=(-6,6))
  last_column = df.iloc[: , -1]
  df = df.drop(columns=df.columns[-1], axis=1)
  scaled = pd.DataFrame(scaler.fit_transform(df.T).T,dtype=object,columns = clmnsws)
  scaled["suggestion"] = last_column
  return scaled
def column_scaler(df):
  scaler = MinMaxScaler(feature_range=(0,1))
  last_column = df.iloc[: , -1]
  df = df.drop(columns=df.columns[-1], axis=1)
  df_scaled = scaler.fit_transform(df.to_numpy())
  df_scaled = pd.DataFrame(df_scaled,columns = clmnsws)
  df_scaled["suggestion"] = last_column
  return df_scaled
def process(df): 
      df = df.dropna()
      df = np.array(df)
      df = extract_data(df)
      df = pd.DataFrame(df,columns = clmns)
      df = row_scaler(df)
      #df = column_scaler(df)
      return df

In [ ]:
def get_data(symbols):
    unattached_dfs = []
    for symbol in symbols:
           data = yf.download(symbol,period="730d",interval="1d",progress=False)
           if data.empty :
              pass
           else:
               if data.shape[0] > 14:
                   unattached_dfs.append(process(data))
    symbols = []
    symbol = []
    data = []

    data = pd.concat(unattached_dfs)
    data = data.astype(float)

    xTrain, xTest, yTrain, yTest = spliting(data)
    print(data.shape)
    data.to_csv(f"{random.randint(1,1000)}.csv")
    data = []
    return xTrain, xTest, yTrain, yTest

#symbols = ["MSFT","AAPL","GOOG","TSLA","AMZN"]
#symbols = ["BTC-USD","LTC-USD","TRX-USD","XRP-USD","ETH-USD","BNB-USD","DASH-USD","VET-USD","LINK-USD","ADA-USD","DOT-USD","SOL-USD","BCH-USD","FTT-USD","FIL-USD","XMR-USD"]
#symbols = ["AAPL","MSFT","TSLA","GOOG"]
#symbols = ["BTC-USD","ETH-USD"]
#symbols = ["BTC-USD"]
#symbols = read_syms_from_txt()
symbols = get_crypto_syms()

#pieces = 10
#new_arrays = np.array_split(symbols, pieces)
#for symbols in new_arrays:
#   get_data(symbols)
    
xTrain, xTest, yTrain, yTest= get_data(symbols)

In [2]:
def spliting(data):
  X = data.drop(["suggestion"],axis=1)
  y = data["suggestion"]
  xTrain, xTest, yTrain, yTest = train_test_split(X, y, test_size=0.1)
  print(xTrain.shape,end=" ")
  print(yTrain.shape)
  print(xTest.shape,end=" ")
  print(yTest.shape)
  return xTrain, xTest, yTrain, yTest

data = pd.read_parquet("/content/drive/MyDrive/lol4.parquet")
data.drop("Unnamed: 0",axis=1,inplace=True)
xTrain, xTest, yTrain, yTest= spliting(data)
data = []

(6087411, 60) (6087411,)
(676380, 60) (676380,)


In [4]:
model = Sequential()



model.add(Dense(2000, activation='relu', input_shape=(xTrain.shape[1],)))
model.add(Dense(2000, activation='relu'))
model.add(Dense(2000, activation='relu'))
model.add(Dense(2000, activation='relu'))
model.add(Dense(2000, activation='relu'))
model.add(Dense(1000, activation='relu'))

model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer="adam", loss='binary_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 2000)              122000    
                                                                 
 dense_8 (Dense)             (None, 2000)              4002000   
                                                                 
 dense_9 (Dense)             (None, 2000)              4002000   
                                                                 
 dense_10 (Dense)            (None, 2000)              4002000   
                                                                 
 dense_11 (Dense)            (None, 2000)              4002000   
                                                                 
 dense_12 (Dense)            (None, 1000)              2001000   
                                                                 
 dense_13 (Dense)            (None, 1)                

In [ ]:
model.fit(xTrain,yTrain,epochs=50,batch_size=20000,validation_data=(xTest,yTest))

Epoch 1/50
305/305 [==============================] - 337s 1s/step - loss: 0.7392 - accuracy: 0.5343 - val_loss: 0.6881 - val_accuracy: 0.5390
Epoch 2/50
305/305 [==============================] - 330s 1s/step - loss: 0.6865 - accuracy: 0.5450 - val_loss: 0.6869 - val_accuracy: 0.5432
Epoch 3/50
305/305 [==============================] - 330s 1s/step - loss: 0.6833 - accuracy: 0.5535 - val_loss: 0.6822 - val_accuracy: 0.5569
Epoch 4/50
305/305 [==============================] - 333s 1s/step - loss: 0.6781 - accuracy: 0.5640 - val_loss: 0.6767 - val_accuracy: 0.5661
Epoch 5/50
305/305 [==============================] - 331s 1s/step - loss: 0.6714 - accuracy: 0.5741 - val_loss: 0.6716 - val_accuracy: 0.5729
Epoch 6/50
305/305 [==============================] - 331s 1s/step - loss: 0.6635 - accuracy: 0.5846 - val_loss: 0.6633 - val_accuracy: 0.5847
Epoch 7/50
305/305 [==============================] - 331s 1s/step - loss: 0.6540 - accuracy: 0.5959 - val_loss: 0.6542 - val_accuracy: 0.5944

In [4]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive
